In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from collections import  Counter
import torch   
#from torchtext import data
import gensim.downloader as api
from sklearn.metrics import fbeta_score
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [64]:
labels = df['Labels'].tolist()
label_combination=[]
label_list=[]

for label in labels:
    num_label=[int(i) for i in label.split()]
    label_list.append(sorted(num_label))
    sort_label=[str(i)for i in sorted(num_label)]
    label_combination.append(' '.join(sort_label))

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(label_list)
y = mlb.transform(label_list)

print(y[:5])
print()
df.Labels.head()

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]]



0         1
1      1 19
2         1
3    8 3 13
4     8 3 7
Name: Labels, dtype: object

In [66]:
df.Labels[1]

'1 19'

In [7]:
reviews_train=df['Caption']
reviews_test = df_test['Caption']
# Remove punctuation/number, and converting to lower cae
import re
def remove_punctuation(x):
    x = re.sub(r'[^a-zA-Z\s]',' ',x.lower())
    # replace more than two continuous space with single space
    # So for the condition of no space between word and punctuation,
    # there is one space between word and punctuation even after punctuation removal
    x =re.sub(' +', ' ', x)
    return x
    
punc_train = [remove_punctuation(s) for s in reviews_train]
punc_test = [remove_punctuation(s) for s in reviews_test]

# Tokenization the texts
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
token_train = [word_tokenize(s) for s in punc_train]
token_test = [word_tokenize(s) for s in punc_test]


nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words('english')

def remove_stopwords(x):
  x = [w for w in x if not w in stop_words]
  return x

stopword_train= [remove_stopwords(s) for s in token_train]
stopword_test= [remove_stopwords(s) for s in token_test]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Parker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Parker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
stopword_test

[['little',
  'girl',
  'waring',
  'krispy',
  'kreme',
  'hat',
  'holding',
  'box',
  'krispy',
  'kreme',
  'donuts'],
 ['beautiful', 'young', 'woman', 'holding', 'orange', 'frisbee'],
 ['group', 'people', 'sitting', 'couch', 'next', 'coffee', 'table'],
 ['person', 'snowboard', 'rides', 'hill'],
 ['man', 'riding', 'skateboard', 'helmet', 'cones'],
 ['group', 'people', 'many', 'suitcases', 'room'],
 ['pink', 'fire', 'hydrant', 'middle', 'yard'],
 ['small', 'boat', 'floating', 'top', 'body', 'water'],
 ['body', 'water', 'sailboat', 'background'],
 ['three', 'yellow', 'boxes', 'filled', 'glazed', 'donuts', 'top', 'table'],
 ['street', 'scene', 'woman', 'crosswalk', 'taxi'],
 ['person', 'riding', 'horse', 'next', 'big', 'black', 'brown', 'dog'],
 ['train', 'train', 'track', 'buildings', 'background'],
 ['man', 'woman', 'pose', 'tennis', 'court'],
 ['balding', 'man', 'standing', 'bed', 'tuck', 'decal'],
 ['man', 'ridding', 'surfboard', 'beach'],
 ['red', 'double', 'decker', 'bus', 'tra

In [15]:
from collections import Counter
train_data = stopword_train
test_data = stopword_test
all_data=train_data+test_data
cpt_len=[len(s) for s in all_data]
seqlen_dict = Counter(cpt_len)
seqlen_dict

Counter({7: 6557,
         4: 5025,
         5: 11733,
         6: 11623,
         9: 904,
         3: 871,
         8: 2626,
         10: 331,
         11: 133,
         13: 35,
         12: 83,
         19: 2,
         17: 6,
         2: 22,
         14: 20,
         16: 8,
         28: 2,
         15: 10,
         18: 3,
         24: 1,
         23: 1,
         22: 1,
         20: 1,
         1: 1,
         21: 1})

In [16]:
max_seqlen = sorted([i for i in seqlen_dict.keys()])[-1]
print('max seqence length: %s' % (max_seqlen))

max seqence length: 28


In [17]:
word_list = [k for i in all_data for k in i]
word_list = list(set(word_list))
max_wordlen = sorted([len(i) for i in word_list])[-1]
print('unique words %d' % (len(word_list)))
print('max word length: %s' % (max_wordlen))

unique words 7838
max word length: 16


In [18]:
info = api.info()  # show info about available models/datasets
model = api.load("glove-twitter-25")  # download the model and return as object ready for use

In [19]:
vec_out = []
for sen in train_data:
    out_temp = []
    for i in range(max_seqlen):
        if i == len(sen):
            for j in range(max_seqlen - i):
                out_temp.append([0]*25)
            break
        else:
            try:
                out_temp.append(model.wv[sen[i]])
            except:
                out_temp.append([0]*25)
    vec_out.append(out_temp)

C:\Users\Parker\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [20]:
text_words_train = np.array(vec_out)
text_words_train.shape

(30000, 28, 25)

In [21]:
vec_out_test = []
for sen in test_data:
    out_temp = []
    for i in range(max_seqlen):
        if i == len(sen):
            for j in range(max_seqlen - i):
                out_temp.append([0]*25)
            break
        else:
            try:
                out_temp.append(model.wv[sen[i]])
            except:
                out_temp.append([0]*25)
    vec_out_test.append(out_temp)

C:\Users\Parker\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [22]:
text_words_test = np.array(vec_out_test)
text_words_test.shape

(10000, 28, 25)

In [24]:
np.save('nlp_embeddings', text_words_train)

In [23]:
np.save('nlp_embeddings_test', text_words_test)

In [26]:
import random
train_data, valid_data, train_label, valid_label = train_test_split(text_words_train, y, test_size=0.2, random_state=9)
print(train_data.shape)
print(valid_data.shape)
print(train_label.shape)
print(valid_label.shape)

(24000, 28, 25)
(6000, 28, 25)
(24000, 18)
(6000, 18)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
#device info
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [28]:
#random generate batches
def generate_batch(input_embeddings,label, batch_size):
    idx = np.random.randint(input_embeddings.shape[0], size=batch_size)
    return input_embeddings[idx,:,:],label[idx,:]

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, batch_first =True,bidirectional=True, dropout=0.2)
        self.linear = nn.Linear(n_hidden*2,n_class)

    def forward(self, sentence):
        
        lstm_out, (h_n,c_n) = self.lstm(sentence)
        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        output = self.linear(hidden_out)
        sig_output = torch.sigmoid(output)
        
        return output, sig_output

In [92]:
learning_rate = 0.005
n_hidden = 32
total_epoch = 8000
n_input = train_data.shape[2]
n_class = train_label.shape[1]
batch_size = 64


In [93]:
# Move the model to GPU
net = Net().to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(total_epoch):
    input_batch, target_batch = generate_batch(train_data,train_label, batch_size)  
    input_batch_torch = torch.from_numpy(input_batch).float().to(device)
    target_batch_torch = torch.from_numpy(target_batch).float().to(device)
     
    net.train()
    outputs, sig_output = net(input_batch_torch)
    loss = criterion(outputs, target_batch_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    net.eval()
    outputs, sig_output = net(input_batch_torch) 
    
    loss = criterion(outputs, target_batch_torch)
    if epoch % 500 == 499:
        #print('Overall F1-Score(samples): ',fbeta_score(target_batch_torch.cpu().detach().numpy(), sig_output.cpu().detach().numpy()>0.5,beta=1, average='samples'))
        print('Epoch: %d, loss: %.5f' %(epoch + 1, loss.item()))
        test(valid_data, valid_label)
print('Finished Training')

Epoch: 500, loss: 0.15129
Overall F1-Score(samples):  0.653497619047619
Epoch: 1000, loss: 0.11534
Overall F1-Score(samples):  0.7477005291005292
Epoch: 1500, loss: 0.11213
Overall F1-Score(samples):  0.7935431216931217
Epoch: 2000, loss: 0.09959
Overall F1-Score(samples):  0.8034959475709474
Epoch: 2500, loss: 0.10835
Overall F1-Score(samples):  0.8177411856661856
Epoch: 3000, loss: 0.09563
Overall F1-Score(samples):  0.8182011063011062
Epoch: 3500, loss: 0.09049
Overall F1-Score(samples):  0.8239740740740742
Epoch: 4000, loss: 0.06456
Overall F1-Score(samples):  0.827385582010582
Epoch: 4500, loss: 0.07520
Overall F1-Score(samples):  0.8325996031746032
Epoch: 5000, loss: 0.11093
Overall F1-Score(samples):  0.8203624098124098
Epoch: 5500, loss: 0.08224
Overall F1-Score(samples):  0.8312643338143337
Epoch: 6000, loss: 0.07389
Overall F1-Score(samples):  0.8316721861471863
Epoch: 6500, loss: 0.07258
Overall F1-Score(samples):  0.8288296777296776
Epoch: 7000, loss: 0.07014
Overall F1-Sco

In [94]:
def test(valid_data, valid_label):
    net.eval()  # set evaluation mode
    valid_data_torch = torch.from_numpy(valid_data).float().to(device)
    with torch.no_grad():
        outputs, sig_output = net(valid_data_torch)
        print('Overall F1-Score(samples): ',fbeta_score(valid_label, sig_output.cpu().detach().numpy()>0.5,beta=1, average='samples'))
    #return pred_result_list, targets

In [95]:
test(valid_data, valid_label)

Overall F1-Score(samples):  0.8310675925925926


In [74]:
def test_label(test_data):
    net.eval()  # set evaluation mode
    test_data_torch = torch.from_numpy(test_data).float().to(device)
    with torch.no_grad():
        outputs, sig_output = net(test_data_torch)
        predicted_labels = np.round(sig_output.cpu().detach().numpy())
        #print('Overall F1-Score(samples): ',fbeta_score(valid_label, sig_output.cpu().detach().numpy()>0.5,beta=1, average='samples'))
    #return pred_result_list, targets
    return predicted_labels

In [75]:
predicted_labels = test_label(text_words_test)

In [76]:
predicted_labels = predicted_labels.astype(int)
predicted_labels = mlb.inverse_transform(predicted_labels)

In [77]:
predicted_labels = [list(i) for i in predicted_labels]

In [97]:
predicted_labels

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [11],
 [9],
 [1, 9],
 [1],
 [1, 3, 10],
 [1, 18],
 [7],
 [1],
 [1],
 [1],
 [1, 3, 6],
 [1],
 [3, 8],
 [1],
 [1],
 [1],
 [1],
 [1],
 [5],
 [17],
 [1],
 [1, 7],
 [1, 8],
 [7],
 [1],
 [5],
 [1],
 [1, 4],
 [1],
 [],
 [3, 8],
 [1],
 [1],
 [17],
 [1],
 [16],
 [1, 7],
 [1],
 [1],
 [1],
 [11, 18],
 [1],
 [1],
 [1],
 [7],
 [1],
 [16],
 [1],
 [1],
 [1],
 [1],
 [15],
 [],
 [1],
 [1],
 [1],
 [1],
 [1, 3, 10],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1, 19],
 [1],
 [1],
 [1, 7],
 [1],
 [17],
 [15, 17],
 [1],
 [1],
 [7],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [3],
 [1, 18],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [3, 8, 10],
 [5],
 [1],
 [1],
 [1],
 [1],
 [3, 10],
 [1],
 [11],
 [1],
 [5],
 [1],
 [1],
 [1],
 [1],
 [15, 17],
 [1],
 [17],
 [1],
 [1],
 [1, 18],
 [1],
 [1],
 [16],
 [17],
 [1, 4],
 [1, 5],
 [1, 3, 4],
 [1, 3],
 [1],
 [1],
 [1],
 [1],
 [1, 18],
 [1],
 [1],
 [9],
 [1],
 [1, 7],
 [15],
 [1],
 [4],
 [1],
 [1],
 [1],
 [11],
 [5],
 [1, 11],
 [1],
 [1],
 [1],
 [1, 19],
 [7

In [84]:
labels = []
for i in predicted_labels:
    temp = ''
    for e in i:
        temp += str(e) + ' '
    temp = temp.rstrip()
    labels.append(temp)

In [86]:
result_temp = pd.DataFrame()
Id = [str(i)+'.jpg' for i in range(30000,40000)]
result_temp['ImageID'] = Id
result_temp['Labels'] = labels
result_temp.to_csv('result_temp.csv', index=False)

In [96]:
result_temp

,ImageID,Labels
0,30000.jpg,1
1,30001.jpg,1
2,30002.jpg,1
3,30003.jpg,1
4,30004.jpg,1
...,...,...
9995,39995.jpg,1
9996,39996.jpg,3 4
9997,39997.jpg,1
9998,39998.jpg,1
